In [123]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from time import time
import pandas as pd
import re
import sys
sys.path.append('../../util/')
import sgt
from collections import Counter

In [124]:
# TODO: add stemming; modify KL code

In [86]:
# create a corpus to give to sklearn
def create_corpus_for_voc(df):
    doc = []
    for i in df.Text.tolist():
        #Remove some non-ascii characters and 'aa's
        i = re.sub(r'aA|aa', 'a', i)
        i = re.sub(r'\\xe2........|\\xc|\\xa|\\n|[0123456789*_]', '', i)
        i = i.lower()
        doc.append(i)  
    return doc

In [94]:
# Get a vocabulary using sklearn's filtering
def get_voc(corpus, ngram, mindf):
    vectorizer = CountVectorizer(stop_words='english', ngram_range=(ngram,ngram),min_df=mindf)
    f = vectorizer.fit_transform(corpus)
    return set(sorted(vectorizer.get_feature_names()))

In [105]:
# compute unigram-frequency dict using the same preprocessing, using only words from the vocabulary
def create_unigram_freq_dict(df):
    text = []
    for line in df.Text.tolist():
        line = re.sub(r'aA|aa', 'a', line)
        line = re.sub(r'\\xe2........|\\xc|\\xa|\\n|[0123456789*_]', '', line).lower()
        line = re.findall(u'(?u)\\b\\w\\w+\\b', line)
        line = [word for word in line if word in vocal]
        text.append(dict(Counter(line)))
    return text

In [95]:
def create_timelist(df):
    timelist = df.PublishDate.drop_duplicates().tolist()
    timelist = [str(i)[:7] for i in timelist]
    return sorted(list(set(timelist)))

In [100]:
def create_df_time(df, time):
    return df[df.PublishDate.str[:7] == time]

In [125]:
# calculate unigram probabilities by simple Good Turing smoothing.
# imput: unigram-freq dict
# output: unigram-prob dict, mimic of a document-term matrix
# if unigram is in this doc, prob = the unigram prob calculated by sgt
# otherwise, prob = the probability given to "all unknown unigrams" by sgt
def calc_sgt(line_dict, voc):
    prob_line = {}
    sgt_line = sgt.simpleGoodTuringProbs(line_dict)
    for word in voc:
        if word in line_dict.keys():
            prob_line[word] = sgt_line[0][word]
        else:
            prob_line[word] = sgt_line[1]
    return prob_line

In [126]:
kl_all = []
t0 = time()
min_df = 3
timelist = create_timelist(df)
for t in timelist:
    df_t = create_df_time(df, t)
    # len(df_t) must > min_df
    # tune this for filtering?
    if len(df_t) > min_df:
        corp = create_corpus_for_voc(df_t)
        vocab = get_voc(corp,1,min_df)
        unigram_dict = create_unigram_freq_dict(df_t)
        for i in unigram_dict:
            print(calc_sgt(i, vocab))
    break
        
#         kl_month = []
#         for row in m.toarray():
#             kl = calc_kl(row, std)
#             if not np.isnan(kl):
#                 kl_month.append(kl)
#         kl_all.append(np.average(kl_month))
# print("done in %0.3fs." % (time() - t0))

p0 = 0.145455
Regression: log(z) = -1.792780*log(r) + 2.487985
{u'dont': 0.046691343314315389, u'just': 0.046691343314315389, u'people': 0.046691343314315389, u'held': 0.01128530849161225, u'want': 0.01128530849161225, u'ground': 0.046691343314315389, u'little': 0.065523294406337265, u'said': 0.23965806652986796, u'away': 0.046691343314315389, u'flying': 0.01128530849161225, u'lot': 0.01128530849161225, u'white': 0.14545454545454545, u'really': 0.01128530849161225, u'way': 0.01128530849161225, u'hand': 0.046691343314315389, u'asked': 0.028352567846030411, u'know': 0.028352567846030411, u'hands': 0.028352567846030411, u'like': 0.065523294406337265, u'youre': 0.028352567846030411, u'theyre': 0.01128530849161225, u'think': 0.01128530849161225}
p0 = 0.113208
Regression: log(z) = -1.637328*log(r) + 2.366825
{u'dont': 0.049937888164122854, u'just': 0.030884315824223137, u'people': 0.0128552654751135, u'held': 0.0128552654751135, u'want': 0.0128552654751135, u'ground': 0.0128552654751135, u'l

In [52]:
df = pd.read_csv('../../data/shakespare_william_works_preprocessed.tsv', sep = '\t')

In [53]:
df = df.head(100)

In [24]:
def calc_kl(p, q):
    return sum([p[i]*(np.log2(p[i]/q[i])) for i in range(len(p))])